# Memory and Context

In [1]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.checkpoint.memory import MemorySaver
import os
from typing import Dict, List, Tuple
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage


In [2]:
# Ensure the OpenAI API key is set
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_KEY")

# 1. Create a memory saver for conversation history
memory = MemorySaver()

# 2. Create chat model
model = ChatOpenAI(temperature=0.7)

# 3. Define the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente conversacional amigable y útil."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

In [3]:
# 4. Define state
def get_state():
    return {"history": []}

# 5. Define the conversation step
def conversation_step(state, input):
    # Get chat history from state
    history = state["history"]
    
    # Format input with history for the model
    messages = prompt.format_messages(history=history, input=input)
    
    # Get response from model
    response = model.invoke(messages)
    
    # Update history with new messages
    new_history = history + [HumanMessage(content=input), response]
    
    # Return updated state
    return {"history": new_history, "response": response.content}

In [4]:
# Initialize state
state = get_state()

# Process first user input
result = conversation_step(state, "Hola, Mi nombre es Kike")
state = {"history": result["history"]}  # Update state
print(result)

# Process follow-up question
result = conversation_step(state, "¿Qué día es hoy?")
# result = conversation_step(state, "¿Quien descibrio el continente americano?")
state = {"history": result["history"]}  # Update state
print(result["response"])

# Process follow-up question
result = conversation_step(state, "¿Recuerdas como me llamo?")
state = {"history": result["history"]}  # Update state
print(result["response"])

{'history': [HumanMessage(content='Hola, Mi nombre es Kike', additional_kwargs={}, response_metadata={}), AIMessage(content='¡Hola Kike! ¡Es un placer conocerte! ¿En qué puedo ayudarte hoy?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 33, 'total_tokens': 53, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Be8652PhYoQNjhDsGssPlrPi5bfzm', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--916225bd-e4e0-4c64-87e1-e23d9c4a036f-0', usage_metadata={'input_tokens': 33, 'output_tokens': 20, 'total_tokens': 53, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})], 'response': '¡Hola Kike! ¡Es un placer cono